In [1]:
import pyvista as pv
import os
import random
import numpy as np

Take N projections of a single random STL file

In [11]:
%%time
# Generate random views of the particle and save as png
save_path = '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/projections'
stl_dir = '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl'
# Get a list of all files in the directory and subdirectories
files = []
for dirpath, _, filenames in os.walk(stl_dir):
    for file in filenames:
        files.append(os.path.join(dirpath, file))  # Store full file paths
random_file = random.choice(files)
stl_path = os.path.join(stl_dir, random_file) # get a random file from stl directory
n_renders = 100
res = 64
bg_color='black'
obj_color='white'
op=1.0
# set plotter 
pl = pv.Plotter(off_screen=True, window_size=[res, res])
pl.background_color = bg_color
pl.enable_parallel_projection()
pl.remove_all_lights()
print(f'reading stl file: {stl_path}')
mesh = pv.read(stl_path) # read stl file as mesh
def random_rotate(mesh):
    rotated = mesh.copy()
    deg_x = np.random.randint(1, 360)
    deg_y = np.random.randint(1, 360)
    deg_z = np.random.randint(1, 360)
    rotated.rotate_x(deg_x, inplace=True)
    rotated.rotate_y(deg_y, inplace=True)
    rotated.rotate_z(deg_z, inplace=True)
    return rotated
count = 0
for i in range(n_renders):
    rotated_mesh = random_rotate(mesh)
    pl.add_mesh(rotated_mesh, show_edges=None, color = obj_color, opacity=op, name='mesh')
    file_name = f'ros_random_rot_{i}.png'
    file_path = os.path.join(save_path, file_name)
    # pl.show(auto_close=False, interactive=False, jupyter_backend='none')
    # pl.show(screenshot=file_path, interactive=False, jupyter_backend='none')
    pl.screenshot(file_path, return_img=False)
    # pl.deep_clean()
    if count % 10==0:
        print(f'progress: {count}/{n_renders}')
    count+=1
pl.close()

reading stl file: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/7/ros-test-030078.stl
progress: 0/100
progress: 10/100
progress: 20/100
progress: 30/100
progress: 40/100
progress: 50/100
progress: 60/100
progress: 70/100
progress: 80/100
progress: 90/100
CPU times: user 934 ms, sys: 44 ms, total: 978 ms
Wall time: 1.16 s


--> Takes ~1 s per 100 projections on single cpu. 

# Scratch

In [2]:
stl_paths_txt = '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/stl_relative_paths.txt'
with open(stl_paths_txt, 'r') as file:
    basepath = '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl'
    rel_paths = [line.strip().replace('./','') for line in file]
    stl_paths = [os.path.join(basepath, i) for i in rel_paths]
print(stl_paths[:10])

['/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-054702.stl', '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-054597.stl', '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-051877.stl', '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-054342.stl', '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-052039.stl', '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-056473.stl', '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-052804.stl', '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-054612.stl', '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-053213.stl', '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-055187.stl']


In [18]:
test_path = stl_paths[0]
print(test_path)

/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-054702.stl


In [17]:
len(stl_paths)

70000

In [22]:
test_path.rsplit('-',1)[1].rsplit('.',1)[0]

'054702'

In [4]:
# Get the total number of tasks (this will be passed by PBS)
num_tasks = 10
# Calculate the chunk size for each task
chunk_size = len(stl_paths) // num_tasks  # Integer division
remainder = len(stl_paths) % num_tasks
print(chunk_size, remainder)

7000 0
